In [ ]:
%pip install pandas
%pip install matplotlib
%pip install seaborn
%pip install scikit-learn

In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline

In [88]:
# Load all datasets
job_postings = pd.read_csv('data/job_postings.csv')
job_skills = pd.read_csv('data/job_details/job_skills.csv')

companies = pd.read_csv('data/company_details/companies.csv')
skills = pd.read_csv('data/maps/skills.csv')

In [89]:
# Check the number of unique values in merging columns for each dataset
unique_values = {
    'job_postings': job_postings['job_id'].nunique(),
    'job_skills': job_skills['job_id'].nunique(),
    'companies': companies['company_id'].nunique(),
    'skills': skills['skill_name'].nunique()
}

unique_values

{'job_postings': 33246, 'job_skills': 32422, 'companies': 11361, 'skills': 35}

In [90]:
# Merging skills and job_skills tables
merged_skill = pd.merge(job_skills,skills, on='skill_abr', how='left')
merged_skill = merged_skill.drop('skill_abr', axis=1)
print(merged_skill)

           job_id              skill_name
0      3690843087     Accounting/Auditing
1      3690843087                 Finance
2      3691763971              Management
3      3691763971           Manufacturing
4      3691775263              Management
...           ...                     ...
56586  3757780487    Health Care Provider
56587  3757934256                  Design
56588  3757934256            Art/Creative
56589  3757934256  Information Technology
56590  3757498232          Administrative

[56591 rows x 2 columns]


In [91]:
# Merge_skills has 56591 rows, but there are only 32422 unique job postings. 
# Therefore we can conclude that there can be 0+ skills per job posting

# Grouping by 'job_id' and concatenating 'skill_name' values
grouped_skills = merged_skill.groupby('job_id')['skill_name'].agg(lambda x: ', '.join(x)).reset_index()

# Display the resulting DataFrame with unique 'job_id' rows and concatenated skills
print(grouped_skills)


           job_id                                    skill_name
0         3958427  Design, Art/Creative, Information Technology
1        85008768                   Sales, Business Development
2       102339515                   Business Development, Sales
3       108965123                                Administrative
4       133114754                   Sales, Business Development
...           ...                                           ...
32417  3757937095                        Information Technology
32418  3757938018                     Management, Manufacturing
32419  3757938019                                   Engineering
32420  3757940025                     Management, Manufacturing
32421  3757940104                                         Other

[32422 rows x 2 columns]


In [92]:
# Merging job postings with corresponding skills
merged_jobs = pd.merge(job_postings, grouped_skills, on='job_id', how='left')
columns_to_drop = ['description', 'max_salary','med_salary','min_salary',
                   'pay_period', 'applies', 'formatted_work_type', 'remote_allowed',
                   'job_posting_url','views','title','application_url','application_type',
                   'expiry','skills_desc','posting_domain','sponsored',
                   'work_type','currency','compensation_type','scraped','closed_time','listed_time'
                   ]
merged_jobs= merged_jobs.drop(columns=columns_to_drop,axis=1)
merged_jobs.head()


,job_id,company_id,location,original_listed_time,formatted_experience_level,skill_name
0,3757940104,553718.0,"Little River, SC",1.699090e+12,Entry level,Other
1,3757940025,2192142.0,"Beaver Dam, WI",1.699080e+12,NaN,"Management, Manufacturing"
2,3757938019,474443.0,"Bessemer, AL",1.699080e+12,NaN,Engineering
3,3757938018,18213359.0,"Aliso Viejo, CA",1.699080e+12,Entry level,"Management, Manufacturing"
4,3757937095,437225.0,United States,1.698970e+12,Mid-Senior level,Information Technology


In [93]:
# Merging Jobs and Companies
merged_data= pd.merge(merged_jobs, companies, on='company_id', how='left')
columns_to_drop = ['description','company_size','zip_code','url','address','state','country','city','company_id']
merged_data= merged_data.drop(columns=columns_to_drop,axis=1)
merged_data.head()

,job_id,location,original_listed_time,formatted_experience_level,skill_name,name
0,3757940104,"Little River, SC",1.699090e+12,Entry level,Other,HearingLife
1,3757940025,"Beaver Dam, WI",1.699080e+12,NaN,"Management, Manufacturing","Metalcraft of Mayville, Inc."
2,3757938019,"Bessemer, AL",1.699080e+12,NaN,Engineering,"U.S. Tsubaki Power Transmission, LLC"
3,3757938018,"Aliso Viejo, CA",1.699080e+12,Entry level,"Management, Manufacturing",Episcopal Communities & Services
4,3757937095,United States,1.698970e+12,Mid-Senior level,Information Technology,"iHerb, LLC"


In [95]:
# Displaying data in a better order
column_order = ['job_id','name','location','skill_name','formatted_experience_level','original_listed_time']
merged_data = merged_data[column_order]
merged_data.head()

,job_id,name,location,skill_name,formatted_experience_level,original_listed_time
0,3757940104,HearingLife,"Little River, SC",Other,Entry level,1.699090e+12
1,3757940025,"Metalcraft of Mayville, Inc.","Beaver Dam, WI","Management, Manufacturing",NaN,1.699080e+12
2,3757938019,"U.S. Tsubaki Power Transmission, LLC","Bessemer, AL",Engineering,NaN,1.699080e+12
3,3757938018,Episcopal Communities & Services,"Aliso Viejo, CA","Management, Manufacturing",Entry level,1.699080e+12
4,3757937095,"iHerb, LLC",United States,Information Technology,Mid-Senior level,1.698970e+12


In [96]:
# Identify missing data in the merged dataset
missing_data = merged_data.isnull().sum()

# Display columns with significant missing data
significant_missing_columns = missing_data[missing_data > 0].sort_values(ascending=False)
significant_missing_columns

formatted_experience_level    9181
skill_name                    1212
name                           705
dtype: int64

In [116]:
# Handling missing values

# Categorical columns with missing values are filled with "Not Specified"
cols_fill_not_specified = ['skill_name', 'name','formatted_experience_level']
for col in cols_fill_not_specified:
    merged_data[col].fillna("Not Specified", inplace=True)

# Check remaining missing values
remaining_missing = merged_data.isnull().sum()
remaining_missing_cols = remaining_missing[remaining_missing > 0].sort_values(ascending=False)
remaining_missing_cols

Series([], dtype: int64)

In [76]:
# Correcting time format of Listed Time
merged_data['original_listed_time'] = merged_data['original_listed_time'].apply(
    lambda x: datetime.datetime.utcfromtimestamp(x / 1000).strftime('%d.%m.%Y'))

C:\Users\Admin\AppData\Local\Temp\ipykernel_15672\3339891841.py:3: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  lambda x: datetime.datetime.utcfromtimestamp(x / 1000).strftime('%d.%m.%Y'))


In [102]:
# Displaying distinct values of Experience_level for categorizing
distinct_values = merged_data['formatted_experience_level'].unique()

print("Distinct values in 'formatted_experience_level':")
for value in distinct_values:
    print(value)


Distinct values in 'formatted_experience_level':
Entry level
Not Specified
Mid-Senior level
Director
Associate
Executive
Internship


In [103]:
# Using map() to categorize column
experience_mapping = {
    'Internship': 1,
    'Entry level': 2,
    'Associate':3, 
    'Mid-Senior level': 4,
    'Director': 5,
    'Executive': 6,
    'Not Specified': 7
}

merged_data['formatted_experience_level'] = merged_data['formatted_experience_level'].map(experience_mapping)

In [104]:
# Pre-processing company name
def clean_company_names(name):
    # Convert to lowercase
    name = name.lower()
    
    # Remove special characters, except spaces, hyphens, and apostrophes
    name = re.sub(r'[^a-zA-Z0-9\s\'-]', '', name)
    
    # Remove extra spaces
    name = re.sub(r'\s+', ' ', name).strip()
    
    # Handle hyphens and apostrophes
    name = name.replace('-', ' ')  # Replace hyphens with spaces
    name = name.replace('\'', '')  # Remove apostrophes
    name = re.sub(r'\s+', ' ', name)

    return name


merged_data['name'] = merged_data['name'].apply(clean_company_names)

In [105]:
# Extract location from City, State format
def extract_city(location):
    words = location.split()
    if len(words) == 1:  # Check if the location has only one word
        return None
    parts = location.split(', ')
    if len(parts) > 1:  # Check if the format is City, State, 
        return parts[0]
    else:
        if "Metropolitan Area" in location:
            return location.split(" Metropolitan Area")[0]
        elif "Area" in location or "Greater" in location:
            return location
        else:
            return None

# Apply the function to the 'location' column
merged_data['location'] = merged_data['location'].apply(extract_city)
merged_data = merged_data.dropna(subset=['location'])

merged_data.shape

(30797, 6)

In [115]:
count_rows = len(merged_data[(merged_data['name'] == 'Not Specified')])
print(f"Number of rows where 'name' is 'Not Specified' and 'location' is None: {count_rows}")

Number of rows where 'name' is 'Not Specified' and 'location' is None: 0


In [81]:
# Remove duplicate rows
merged_data_cleaned = merged_data.drop_duplicates()

# Shape of the cleaned data
merged_data_cleaned.shape

(30797, 6)

In [82]:
merged_data_cleaned = merged_data_cleaned.rename(columns={'name':'company','skill_name':'required_skill',
                                                           'formatted_experience_level':'experience_level',
                                                           'original_listed_time':'day_posted'})

merged_data_cleaned = merged_data_cleaned.sort_values('job_id', ascending=False)


In [83]:
merged_data_cleaned


,job_id,company,location,required_skill,experience_level,day_posted
0,3757940104,hearinglife,Little River,Other,2,04.11.2023
1,3757940025,metalcraft of mayville inc,Beaver Dam,"Management, Manufacturing",7,04.11.2023
2,3757938019,us tsubaki power transmission llc,Bessemer,Engineering,7,04.11.2023
3,3757938018,episcopal communities services,Aliso Viejo,"Management, Manufacturing",2,04.11.2023
6,3757937004,boyd group services inc,Daytona Beach,"Management, Manufacturing",2,04.11.2023
...,...,...,...,...,...,...
33241,133114754,cargologin,Santa Clarita,"Sales, Business Development",7,23.08.2023
33242,108965123,not specified,New York,Administrative,7,03.11.2023
33243,102339515,dryerventz ductventz,Greater Boston,"Business Development, Sales",7,03.11.2023
33244,85008768,not specified,Chico,"Sales, Business Development",7,23.08.2023


In [84]:
# Group by 'location' column and count occurrences
city_counts = merged_data_cleaned['location'].value_counts()

# Sort the cities by their occurrence counts in descending order
sorted_cities = city_counts.sort_values(ascending=False)

# Display the sorted cities with their occurrence counts
print(sorted_cities)

location
New York                     1004
Chicago                       535
Los Angeles                   487
Atlanta                       447
Houston                       446
                             ... 
Los Altos                       1
Purcellville                    1
Hernando                        1
Beaumont-Port Arthur Area       1
Little River                    1
Name: count, Length: 3742, dtype: int64
